In [4]:
# Mapillary Vistas -> YOLO (8 utilities; training split)
# - Reads:   <ROOT>\training\images\*.jpg|png and <ROOT>\training\(v2.0\)polygons\*.json
# - Writes:  <OUT_BASE>\images\train , <OUT_BASE>\labels\train , <OUT_BASE>\data.yaml
# - Classes / IDs: 0 fire_hydrant, 1 junction_box, 2 manhole, 3 parking_meter,
#                  4 water_valve, 5 traffic_sign, 6 traffic_light, 7 utility_pole

import os, json, shutil
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import numpy as np
from collections import Counter, defaultdict

# ====== EDIT THESE TWO PATHS ======
ROOT = Path(r"C:\Users\GLazar\Downloads\Mapillary Vistas\v2.0")
OUT_BASE = Path(r"C:\Users\GLazar\Downloads\Mapillary Vistas\YOLO_Utilities_TrainOnly")
# ==================================

SPLIT = "validation"    # use "validation" if you want val instead
IOU_THRESH = 0.5      # per-class NMS inside each image

# ---- EXACT ORDER YOU REQUESTED ----
CLASS_NAMES = [
    "fire_hydrant",   # 0
    "junction_box",   # 1
    "manhole",        # 2
    "parking_meter",  # 3
    "water_valve",    # 4
    "traffic_sign",   # 5
    "traffic_light",  # 6
    "utility_pole",   # 7
]
NAME_TO_ID = {n:i for i,n in enumerate(CLASS_NAMES)}

# Mapillary label -> our class name (ONLY these 8 kept)
LABEL_MAP = {
    # Utilities
    "object--fire-hydrant": "fire_hydrant",
    "object--junction-box": "junction_box",
    "object--manhole": "manhole",
    "object--parking-meter": "parking_meter",
    "object--water-valve": "water_valve",

    # Traffic lights (all combined)
    "object--traffic-light--general-single": "traffic_light",
    "object--traffic-light--pedestrians": "traffic_light",
    "object--traffic-light--general-upright": "traffic_light",
    "object--traffic-light--general-horizontal": "traffic_light",
    "object--traffic-light--cyclists": "traffic_light",
    "object--traffic-light--other": "traffic_light",

    # Traffic signs / signage (all combined)
    "object--traffic-sign--front": "traffic_sign",
    "object--traffic-sign--direction-front": "traffic_sign",
    "object--traffic-sign--information-parking": "traffic_sign",
    "object--traffic-sign--temporary-front": "traffic_sign",
    "object--traffic-sign--ambiguous": "traffic_sign",
    "object--traffic-sign--back": "traffic_sign",
    "object--traffic-sign--direction-back": "traffic_sign",
    "object--traffic-sign--temporary-back": "traffic_sign",
    "object--sign--information": "traffic_sign",
    "object--sign--other": "traffic_sign",
    "object--sign--ambiguous": "traffic_sign",
    "object--sign--back": "traffic_sign",

    # Poles (ONLY utility_pole kept)
    "object--support--utility-pole": "utility_pole",
}

def ensure_dirs():
    (OUT_BASE/"images"/"val").mkdir(parents=True, exist_ok=True)
    (OUT_BASE/"labels"/"val").mkdir(parents=True, exist_ok=True)

def find_paths(split=SPLIT):
    img_dir = ROOT/split/"images"
    poly_dir = ROOT/split/"v2.0"/"polygons"
    if not poly_dir.exists():
        poly_dir = ROOT/split/"polygons"  # fallback for older layouts
    return img_dir, poly_dir

def iou_xyxy(a, b):
    x1 = max(a[0], b[0]); y1 = max(a[1], b[1])
    x2 = min(a[2], b[2]); y2 = min(a[3], b[3])
    iw = max(0.0, x2 - x1); ih = max(0.0, y2 - y1)
    inter = iw * ih
    if inter <= 0: return 0.0
    area_a = (a[2]-a[0])*(a[3]-a[1]); area_b = (b[2]-b[0])*(b[3]-b[1])
    return inter / max(1e-9, (area_a + area_b - inter))

def nms_per_class(boxes, iou_thr=IOU_THRESH):
    if not boxes: return []
    areas = [(b[2]-b[0])*(b[3]-b[1]) for b in boxes]
    order = np.argsort(areas)[::-1]
    keep, used = [], [False]*len(order)
    for idx_i, i in enumerate(order):
        if used[idx_i]: continue
        bi = boxes[i]
        keep.append(bi)
        for idx_j, j in enumerate(order[idx_i+1:], start=idx_i+1):
            if used[idx_j]: continue
            if iou_xyxy(bi, boxes[j]) >= iou_thr:
                used[idx_j] = True
    return keep

def poly_to_xyxy(poly, W, H):
    coords = np.asarray(poly, dtype=float)
    if coords.ndim != 2 or coords.shape[1] < 2:
        return None
    x1 = float(np.clip(coords[:,0].min(), 0, W))
    x2 = float(np.clip(coords[:,0].max(), 0, W))
    y1 = float(np.clip(coords[:,1].min(), 0, H))
    y2 = float(np.clip(coords[:,1].max(), 0, H))
    if x2 - x1 <= 1 or y2 - y1 <= 1:
        return None
    return [x1,y1,x2,y2]

def xyxy_to_yolo(xyxy, W, H):
    x1,y1,x2,y2 = xyxy
    xc = ((x1+x2)/2.0)/W
    yc = ((y1+y2)/2.0)/H
    wn = (x2-x1)/W
    hn = (y2-y1)/H
    return xc,yc,wn,hn

def convert_split_to_train():
    img_dir, poly_dir = find_paths(SPLIT)
    out_img = OUT_BASE/"images"/"val"
    out_lbl = OUT_BASE/"labels"/"val"

    if not img_dir.exists() or not poly_dir.exists():
        raise FileNotFoundError(f"Missing paths:\n  images={img_dir}\n  polygons={poly_dir}")

    stats_imgs = 0
    stats_boxes = Counter()

    jsons = sorted(poly_dir.glob("*.json"))
    for jpath in tqdm(jsons, desc=f"Converting {SPLIT} -> train"):
        stem = jpath.stem

        # find image
        img_path = None
        for ext in (".jpg",".png",".jpeg"):
            p = img_dir/f"{stem}{ext}"
            if p.exists():
                img_path = p; break
        if img_path is None:
            continue

        # read polygons
        try:
            data = json.loads(jpath.read_text())
        except Exception:
            continue
        objs = data.get("objects") or []
        if not objs:
            continue

        # image size
        try:
            with Image.open(img_path) as im:
                W,H = im.size
        except Exception:
            continue

        boxes_by_cls = defaultdict(list)

        for o in objs:
            lbl = o.get("label")
            if lbl not in LABEL_MAP:   # drop everything else
                continue
            cls_name = LABEL_MAP[lbl]
            cls_id = NAME_TO_ID[cls_name]
            poly = o.get("polygon")
            if not isinstance(poly, list) or len(poly) == 0:
                continue
            xyxy = poly_to_xyxy(poly, W, H)
            if xyxy is None:
                continue
            boxes_by_cls[cls_id].append(xyxy)

        if not boxes_by_cls:
            continue

        # per-class NMS and write
        lines = []
        for cls_id, boxes in boxes_by_cls.items():
            kept = nms_per_class(boxes)
            stats_boxes[cls_id] += len(kept)
            for b in kept:
                xc,yc,wn,hn = xyxy_to_yolo(b, W, H)
                lines.append(f"{cls_id} {xc:.6f} {yc:.6f} {wn:.6f} {hn:.6f}")

        if not lines:
            continue

        # write label + copy image
        (out_lbl/f"{stem}.txt").write_text("\n".join(lines) + "\n")
        shutil.copyfile(img_path, out_img/img_path.name)
        stats_imgs += 1

    # data.yaml
    (OUT_BASE/"data.yaml").write_text(
        "path: " + str(OUT_BASE).replace("\\", "/") + "\n"
        "train: images/train\n"
        "val: images/train\n"  # change later if you create a real val split
        f"nc: {len(CLASS_NAMES)}\n"
        f"names: {CLASS_NAMES}\n"
    )

    print("\n=== Export summary (train only) ===")
    print("Images (train):", stats_imgs)
    print("Objects per class:")
    for i,name in enumerate(CLASS_NAMES):
        print(f"  {i:02d} {name:<14} {stats_boxes[i]}")

if __name__ == "__main__":
    ensure_dirs()
    convert_split_to_train()
    print(f"\n✅ Done. Dataset at: {OUT_BASE}\n   data.yaml -> {OUT_BASE/'data.yaml'}")


Converting validation -> train: 100%|█████████████████████████████████████████████| 2000/2000 [00:11<00:00, 172.54it/s]


=== Export summary (train only) ===
Images (train): 1959
Objects per class:
  00 fire_hydrant   211
  01 junction_box   636
  02 manhole        1110
  03 parking_meter  95
  04 water_valve    150
  05 traffic_sign   18456
  06 traffic_light  7590
  07 utility_pole   6713

✅ Done. Dataset at: C:\Users\GLazar\Downloads\Mapillary Vistas\YOLO_Utilities_TrainOnly
   data.yaml -> C:\Users\GLazar\Downloads\Mapillary Vistas\YOLO_Utilities_TrainOnly\data.yaml
